In [1]:
import pennylane as qml

In [122]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow import compat

In [256]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [257]:
#Load MNIST datasets from Keras
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

#Rescale the images from [0, 255] to the [0.0, 1.0] range.
train_images, test_images = train_images[..., np.newaxis]/255.0, test_images[..., np.newaxis]/255.0

In [258]:
train_filter_tf0 = np.where(train_labels == 0)
train_filter_tf1 = np.where(train_labels == 1)

train_images_tf0, train_labels_tf0 = train_images[train_filter_tf0], train_labels[train_filter_tf0]

train_images_tf1, train_labels_tf1 = train_images[train_filter_tf1], train_labels[train_filter_tf1]

In [259]:
train_filter_tf = np.where((train_labels == 0 ) | (train_labels == 1 )| (train_labels == 2 )
                          | (train_labels == 3 )| (train_labels == 4 )| (train_labels == 5 )
                          | (train_labels == 6 )| (train_labels == 7 ))
test_filter_tf = np.where((test_labels == 0 ) | (test_labels == 1 )| (test_labels == 2 )
                         | (test_labels == 3 )| (test_labels == 4 )| (test_labels == 5 )
                         | (test_labels == 6 )| (test_labels == 7 ))

In [260]:
train_images_tf, train_labels_tf = train_images[train_filter_tf], train_labels[train_filter_tf]
test_images_tf, test_labels_tf = test_images[test_filter_tf], test_labels[test_filter_tf]

In [261]:
train_labels_tf = keras.utils.to_categorical(train_labels_tf, num_classes-2)

In [262]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes-2, activation="softmax"),
    ]
)

model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_15 (Flatten)         (None, 1600)              0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 1600)              0         
_________________________________________________________________
dense_23 (Dense)             (None, 8)               

In [263]:
batch_size = 128
epochs = 10

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(train_images_tf, train_labels_tf, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/10
339/339 [==============================] - 14s 42ms/step - loss: 0.3024 - accuracy: 0.9098 - val_loss: 0.0547 - val_accuracy: 0.9857
Epoch 2/10
339/339 [==============================] - 17s 49ms/step - loss: 0.0766 - accuracy: 0.9763 - val_loss: 0.0411 - val_accuracy: 0.9900
Epoch 3/10
339/339 [==============================] - 13s 39ms/step - loss: 0.0566 - accuracy: 0.9817 - val_loss: 0.0371 - val_accuracy: 0.9913
Epoch 4/10
339/339 [==============================] - 17s 50ms/step - loss: 0.0490 - accuracy: 0.9848 - val_loss: 0.0317 - val_accuracy: 0.9923
Epoch 5/10
339/339 [==============================] - 15s 43ms/step - loss: 0.0426 - accuracy: 0.9866 - val_loss: 0.0313 - val_accuracy: 0.9927
Epoch 6/10
339/339 [==============================] - 15s 46ms/step - loss: 0.0374 - accuracy: 0.9879 - val_loss: 0.0270 - val_accuracy: 0.9929
Epoch 7/10
339/339 [==============================] - 14s 42ms/step - loss: 0.0362 - accuracy: 0.9888 - val_loss: 0.0247 - val_accuracy:

In [264]:
extractor = keras.Model(inputs=model.inputs,
                        outputs=[layer.output for layer in model.layers])
features0 = extractor(train_images_tf0)
features1 = extractor(train_images_tf1)

In [265]:
tf.compat.v1.enable_eager_execution()

In [266]:
X_train_0 = np.array(tf.constant(features0[6]))
X_train_1 = np.array(tf.constant(features1[6]))

In [267]:
X_train_0 = X_train_0[:100]
X_train_1 = X_train_1[:100]

In [268]:
#Re-scale 
def rescale_linear(array, new_min, new_max):
    """Rescale an arrary linearly."""
    minimum, maximum = np.min(array), np.max(array)
    m = (new_max - new_min) / (maximum - minimum)
    b = new_min - m * minimum
    return m * array + b

In [269]:
X_train_0[50]

array([9.9996591e-01, 1.6879586e-11, 2.8085671e-05, 9.0969227e-10,
       8.4807725e-09, 3.9191441e-08, 5.9049908e-06, 5.2797532e-08],
      dtype=float32)

In [270]:
X_train_1[50]

array([9.2606166e-07, 9.9975532e-01, 6.5584121e-05, 5.5110820e-07,
       1.7101977e-04, 4.0437637e-08, 9.6511599e-07, 5.6995118e-06],
      dtype=float32)

In [271]:
print(rescale_linear(X_train_1[0], 0, np.pi/2))

[6.4107991e-07 1.5707964e+00 1.9779854e-05 1.0654456e-07 1.2680655e-04
 0.0000000e+00 8.3861460e-06 5.7085465e-05]


In [272]:
X_train_1[3]

array([2.0722359e-07, 9.9989200e-01, 2.2830254e-05, 4.4897016e-07,
       6.7050292e-05, 5.4192692e-06, 9.1852326e-07, 1.1257809e-05],
      dtype=float32)

In [273]:
#Final Dataset
X_train_0_f = []
X_train_1_f = []

for i in range(100):
    X_train_0_f.append(rescale_linear(X_train_0[i], 0, np.pi/2))
    X_train_1_f.append(rescale_linear(X_train_1[i], 0, np.pi/2))

In [274]:
type(X_train_0_f[0])

numpy.ndarray

In [275]:
len(X_train_0_f)

100

In [276]:
num_wires = 8

In [277]:
def qubit_encoding(vector):
    for i in range(num_wires):
        qml.RY(vector[i]*2, wires=i)

In [278]:
def U_block_t(params, wires):
    """
    2 qubit block representing U(D) blocks in the paper that has control wire on the top wire
    """
    qml.RY(params[0], wires=wires[0])
    qml.RY(params[1], wires=wires[1])
    qml.CNOT(wires=[wires[0], wires[1]])

In [279]:
def U_block_b(params, wires):
    """
    2 qubit block represetning U(D) blocks in the paper that has control wire is on the bottom wire
    """
    qml.RY(params[0], wires=wires[0])
    qml.RY(params[1], wires=wires[1])
    qml.CNOT(wires=[wires[1],wires[0]])

In [280]:
qml.enable_tape()
dev1 = qml.device('default.qubit', wires=num_wires)

@qml.qnode(dev1, diff_method="backprop")
def variational_circuit(params, features):
    #qubit_encoding
    qubit_encoding(features)
    
    #MERA circuit
    #D blocks on (1,2), (3,4), (5,6), (7,8)    
    U_block_t(params[:2],wires=[1,2])
    U_block_b(params[2:4],wires=[3,4])
    U_block_t(params[4:6],wires=[5,6])
    #U_block_t(params[6:8],wires=[7,8])

    #U blocks on (0,1), (2,3), (4,5), (6,7), (8,9)
    U_block_t(params[6:8],wires=[0,1])
    U_block_d(params[8:10],wires=[2,3])
    U_block_t(params[10:12],wires=[4,5])
    U_block_d(params[12:14],wires=[6,7])
    #U_block_d(params[:2],wires=[1,2])
    
    #D in the middle
    U_block_t(params[14:16],wires=[2,5])
    
    #Two U's
    U_block_t(params[16:18],wires=[1,2])
    U_block_d(params[18:20],wires=[5,6])
    
    #Last U
    U_block_t(params[20:22],wires=[2,5])
    
    return qml.expval(qml.PauliZ(5))

In [281]:
def square_loss(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

def cost(var, X, Y):
    predictions = [variational_circuit(var, x) for x in X]
    return square_loss(Y, predictions)

In [282]:
X_train = X_train_0_f + X_train_1_f

In [283]:
Y_train = train_labels_tf0[:100].tolist() + train_labels_tf1[:100].tolist()

In [285]:
np.random.seed(0)
var_init = np.random.randn(22)

opt = qml.NesterovMomentumOptimizer(0.5)
batch_size = 20

#optimization
var = var_init
for it in range(50):

    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, len(X_train), (batch_size,))
    X_batch = [X_train[i] for i in batch_index]
    Y_batch = [Y_train[i] for i in batch_index]
    var, cost_new = opt.step_and_cost(lambda v: cost(v, X_batch, Y_batch), var)
    print(cost_new)

0.46344729527647344
0.21083671441965599
0.21371911430833496
0.13615002634894263
0.04589539069405711
0.015281679230924891
0.0037935259792574462
0.010990378215842832
0.012320800483042476
0.013614789424986447
0.0073969691970388555
0.004554985284519623
0.007493186547530952
0.002329906481891377
0.0023732308383424363
0.0021002160885410363
0.0010128128343910305
0.0011539128563409396
0.0009829709065186669
0.0009481901009537665
0.0012725030981922713
0.0012168343621481018
0.0012330197671389174
0.0020099544451545833
0.0014222170335035124
0.0010134665035070756
0.0014747898340482111
0.0008785614531273122
0.0009441842247379939
0.0009232987228231957
0.0006445090539631438
0.0006393413878367942
0.0011862795487816546
0.0013555296413589348
0.00047378369162995125
0.0009636250885353609
0.00013944673575877996
0.00019542178637506469
0.00018072310170021036
0.0002041584421871986
0.0001312843240345654
8.218577446501196e-05
0.00011272309686164101
0.0008344564992297784
0.00038159361314585365
8.939419193659715e-05

In [286]:
var_init

array([ 1.76405235,  0.40015721,  0.97873798,  2.2408932 ,  1.86755799,
       -0.97727788,  0.95008842, -0.15135721, -0.10321885,  0.4105985 ,
        0.14404357,  1.45427351,  0.76103773,  0.12167502,  0.44386323,
        0.33367433,  1.49407907, -0.20515826,  0.3130677 , -0.85409574,
       -2.55298982,  0.6536186 ])

In [287]:
var

tensor([ 1.97691224, -1.14761573,  0.7780793 ,  2.99357889,  2.33036416,
        -0.7489642 ,  0.93982075,  0.07013816,  0.82772584,  0.85359053,
         0.27173761,  1.35890477,  0.64816301, -0.00724718, -0.16305724,
         1.11857802,  1.60143486, -1.96183988,  0.08322165, -0.98074687,
        -2.30456912,  0.86276042], requires_grad=True)

In [289]:
#Create X_test
test_filter_tf0 = np.where(test_labels == 0)
test_filter_tf1 = np.where(test_labels == 1)

test_images_tf0, test_labels_tf0 = test_images[test_filter_tf0], test_labels[test_filter_tf0]

test_images_tf1, test_labels_tf1 = test_images[test_filter_tf1], test_labels[test_filter_tf1]

In [290]:
features0_test = extractor(test_images_tf0)
features1_test = extractor(test_images_tf1)

In [291]:
X_test_0 = np.array(tf.constant(features0_test[6]))
X_test_1 = np.array(tf.constant(features1_test[6]))

In [292]:
X_test_0 = X_train_0[:100]
X_test_1 = X_train_1[:100]

In [293]:
#Final Test Dataset
X_test_0_f = []
X_test_1_f = []

for i in range(100):
    X_test_0_f.append(rescale_linear(X_test_0[i], 0, np.pi/2))
    X_test_1_f.append(rescale_linear(X_test_1[i], 0, np.pi/2))

In [294]:
X_test = X_test_0_f + X_test_1_f
Y_test = test_labels_tf0[:100].tolist() + test_labels_tf1[:100].tolist()

In [301]:
label_results=[]
for x in X_test:
    #print(variational_classifier(var,x))
    if 0.5 > np.abs(variational_classifier(var, x)):
        label_results.append(0)
    else:
        label_results.append(1)

In [302]:
print(label_results)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
